In [ ]:
!pip install google-cloud-pubsub==2.26.1

In [ ]:
PROJECT_ID = "replace_with_project_id" # @param {"type":"string"}
TOPIC_NAME = "replace_with_pubsub_topic_name" # @param {"type":"string"}


In [ ]:
import json
import logging
import os
import time
import datetime

from google.cloud import pubsub_v1
from concurrent import futures


# Function to generate a single customer record
def generate_customer_record():
    return {
        "customer_id": 1
    }

# Function to generate multiple customer records
def generate_customer_records(num_records):
    for r in range(num_records):
      yield generate_customer_record()


logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

topic_name = 'projects/{project_id}/topics/{topic}'.format(
    project_id=PROJECT_ID,
    topic=TOPIC_NAME,
)

# !!DO NOT USE THESE SETTINGS FOR PRODUCTION, THESE ARE SPECIFICALLY DESIGNED TO RESTRICT MESSAGE FLOW
batch_settings = pubsub_v1.types.BatchSettings(
    max_messages=10, # Send immediately when 10 messages are published
    max_bytes=1024,  # Send immediately when 1 KB is reached
    max_latency=0.1  # Send every 0.1 seconds
)

# !!DO NOT USE THESE SETTINGS FOR PRODUCTION, THESE ARE SPECIFICALLY DESIGNED TO RESTRICT MESSAGE FLOW
publisher_options = pubsub_v1.types.PublisherOptions(
    flow_control = pubsub_v1.types.PublishFlowControl(
      message_limit=10, # Only 10 messages allowed in memory buffer
      byte_limit=1024,  # Only 1 KB allowed in memory buffer
      limit_exceeded_behavior=pubsub_v1.types.LimitExceededBehavior.BLOCK #!!!
  )
)
publisher = pubsub_v1.PublisherClient(
    batch_settings=batch_settings,
    publisher_options=publisher_options
)

published_futures = []

total_should_be_sent = 0
start_time = time.time()

for r in generate_customer_records(10000): # sending 10k messages
    future = publisher.publish(topic_name, json.dumps(r).encode('utf-8'), origin='webapp_a', user="moo")
    published_futures.append(future)
    total_should_be_sent += 1

futures.wait(published_futures, return_when=futures.ALL_COMPLETED)

end_time = time.time()
elapsed_time = end_time - start_time

# Wait for all publishing futures to complete and handle any exceptions
for future in published_futures:
    try:
        result = future.result()  # This will raise an exception if publishing failed
    except Exception as e:
        raise e

logging.info(f"Published messages with batch settings to {topic_name}.")
logging.info(f"Total should be sent: {total_should_be_sent}.")

logging.info(f"Code ran in {elapsed_time:.4f} seconds.")